# Load data

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from gensim import corpora
import gensim
import nltk
import string
import spacy
import en_core_web_sm


In [ ]:
filepath = 'query4/'
filename = 'anemia_and_pain_and_opioid.csv'
dataframe_1 = pd.read_csv(filepath+filename)
#print('No.of records:', len(dataframe_1))
#print(type(dataframe_1))
filename_2= 'anemia_or_pain_or_opioid.csv'
dataframe_2=pd.read_csv(filepath+filename_2)
print('No.of records:', len(dataframe_2))
print(type(dataframe_2))
#dataframe=dataframe_1.join(dataframe_2)
frames=[dataframe_1,dataframe_2]
dataframe=pd.concat(frames, axis=0, join='outer', ignore_index=False)
dataframe=dataframe.reset_index()
print(len(dataframe))
#for index,row in dataframe.iterrows():
#    print(row)


In [ ]:
#en_core_web_sm.load()
import re
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    #print(text)
    tokens = parser(text)
    stop_list=['<','>','</i>','<i>','<b>','</b>','=','<i','<b','</i','</b','<sub>','</sub>','<sub']
    parser.Defaults.stop_words.update(stop_list)
    #print(tokens)
    #[token.text for token in doc if token.is_stop != True and token.is_punct != True]
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.is_stop==True:
            continue
        elif re.search(r'[a-zA-z0-9]*<\/[a-z]+',str(token)):
            continue
        elif token.is_punct==True:
            continue
        else:
            lda_tokens.append(token.lower_)
    #print(lda_tokens)
    return lda_tokens 

In [4]:
nltk.download('wordnet')
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def get_lemma2(word):
    lemma = wordnet.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def get_lemma(word):
    #print(WordNetLemmatizer().lemmatize(word))
    lemma = wordnet.morphy(word)
    if lemma is None:
        y=word
    else:
        y=lemma
    x=lemmatizer.lemmatize(y, get_wordnet_pos(word))
    #print(word,y,x)
    return x
    #return WordNetLemmatizer().lemmatize(word)

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [5]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 11001] getaddrinfo failed>


In [6]:
def prepare_text_for_lda(text):
    # convert all words into lower case, split by white space
    #tokens = text.strip().split()
    #  remove words with 1 or 2 letters (small words, punctuation)
    #tokens = [token for token in tokens if len(token) > 2]
    # remove English stopwords (as defined by NLTK)
    #tokens = [token for token in tokens if token not in en_stop]
    #return tokens
    tokens = tokenize(str(text))
    tokens = [token for token in tokens if len(token) > 4]
    #tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    #print(tokens)
    return tokens

In [7]:
titles = []
abstracts=[]
abstract_text_data=[]
title_text_data=[]
for index, row in dataframe.iterrows():
    #Preparing abstract text for LDA 
    abstract = row['Abstract']
    #abstract= "".join([char for char in abstract_text if char not in string.punctuation])
    abstracts.append(abstract)
    tokens = prepare_text_for_lda(abstract)
    abstract_text_data.append(tokens)
    
    #Preparing title text for LDA
    title= row['Title']
    #title="".join([char for char in title_text if char not in string.punctuation])
    #print(title)
    titles.append(title)
    title_tokens=prepare_text_for_lda(title)
    title_text_data.append(title_tokens) 
    #print(index)
    if(index==299):
        print("BREAK")
        break
#print(titles)
#print(len(abstracts))
print(len(title_text_data))
print(len(abstract_text_data))



BREAK
300
300


## Creating LDA model for abstracts

## Creating corpus and dicitionary from abstract data 

In [8]:
from gensim import corpora
dictionary = corpora.Dictionary(abstract_text_data)
#print(dictionary)
corpus = [dictionary.doc2bow(text) for text in abstract_text_data]
import pickle
#pickle.dump(corpus, open(filepath+filename+'_corpus.pkl', 'wb'))
#dictionary.save(filepath+filename+'_dictionary.gensim')
print(len(corpus))

300


## Forming the base LDA model

In [9]:
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary,passes=10)
#ldamodel.save(filepath+filename+'_abstract.gensim')
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.044*"opioid" + 0.021*"patient" + 0.017*"analgesic" + 0.012*"abuse" + 0.011*"prescription"')
(1, '0.023*"patient" + 0.014*"sickle" + 0.014*"opioid" + 0.013*"morphine" + 0.012*"disease"')
(2, '0.034*"patient" + 0.015*"opioid" + 0.012*"morphine" + 0.012*"study" + 0.012*"sickle"')
(3, '0.030*"opioid" + 0.020*"analgesic" + 0.012*"patient" + 0.010*"relate" + 0.008*"study"')
(4, '0.022*"patient" + 0.020*"sickle" + 0.020*"opioid" + 0.018*"opioids" + 0.014*"management"')


## Coherence score for base LDA model

In [11]:
from gensim.models import CoherenceModel# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=abstract_text_data, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.38718672259698456


In [10]:
best_ldamodel = gensim.models.LdaMulticore(corpus,num_topics=7, 
                                           random_state=100,
                                           passes=10,
                                           alpha=0.90,
                                           eta=0.61,id2word=dictionary)
#ldamodel.save(filepath+filename+'_abstract.gensim')
best_topics = best_ldamodel.print_topics(num_words=5)
for topic in best_topics:
    print(topic)

(0, '0.003*"follow" + 0.003*"orthopedic" + 0.003*"trauma" + 0.003*"postoperative" + 0.002*"prolong"')
(1, '0.019*"patient" + 0.015*"sickle" + 0.010*"morphine" + 0.008*"disease" + 0.008*"treatment"')
(2, '0.015*"patient" + 0.008*"disease" + 0.007*"sickle" + 0.007*"opioid" + 0.006*"high"')
(3, '0.026*"opioid" + 0.017*"opioids" + 0.015*"analgesic" + 0.012*"patient" + 0.012*"chronic"')
(4, '0.033*"opioid" + 0.018*"analgesic" + 0.016*"patient" + 0.011*"prescribe" + 0.007*"adverse"')
(5, '0.023*"opioid" + 0.019*"patient" + 0.011*"visit" + 0.009*"opioids" + 0.008*"admission"')
(6, '0.019*"sickle" + 0.018*"patient" + 0.017*"disease" + 0.013*"opioid" + 0.009*"analgesic"')


In [12]:
from gensim.models import CoherenceModel# Compute Coherence Score
best_coherence_model_lda = CoherenceModel(model=best_ldamodel, texts=abstract_text_data, dictionary=dictionary, coherence='c_v')
best_coherence_lda = best_coherence_model_lda.get_coherence()
print('\nCoherence Score: ', best_coherence_lda)



Coherence Score:  0.40565117284501484


In [13]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(best_ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_display, filepath+'abstract_anemia_pain_opioid.html')
pyLDAvis.display(lda_display)


## Coherence scores for various values of a,b and k

In [ ]:
from gensim.models import CoherenceModel# Compute Coherence Score

def compute_coherence_values(corpus,data, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
import numpy as np
import tqdm
grid = {}
grid['Validation_Set'] = {}# Topics range
min_topics = 5
max_topics = 9
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')


In [ ]:
# Validation sets
#num_of_docs = len(corpus)
#corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               #gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
               #corpus]
#corpus_title = ['75% Corpus', '100% Corpus']
model_results = {#'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
}# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=120)
    
    # iterate through validation corpuses
    #for i in range(len(corpus_sets)):
        # iterate through number of topics
    for k in topics_range:
            # iterate through alpha values
        for a in alpha:
                # iterare through beta values
            for b in beta:
                    # get the coherence score for the given parameters
                cv = compute_coherence_values(corpus=corpus,data=abstract_text_data, dictionary=dictionary,k=k, a=a, b=b)
                    # Save the model results
                   # model_results['Validation_Set'].append(corpus_title[i])
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)
                    
                pbar.update(1)
    #pd.DataFrame(model_results).to_csv('lda_tuning_query_and_or_opioid.csv', index=False)
    pbar.close()

In [ ]:
pd.DataFrame(model_results).to_csv(filepath+'abstract_query4_and_or_opioid_0.3.csv', index=False)

## Creating LDA model for titles

In [19]:
from gensim import corpora
dictionary_title = corpora.Dictionary(title_text_data)
corpus_title = [dictionary_title.doc2bow(text) for text in title_text_data]
import pickle
#pickle.dump(corpus_title, open(filepath+filename+'_corpus_title.pkl', 'wb'))
#dictionary.save(filepath+filename+'_dictionary_title.gensim')

In [20]:
NUM_TOPICS = 5
ldamodel_title = gensim.models.ldamodel.LdaModel(corpus_title, num_topics = NUM_TOPICS, id2word=dictionary_title, passes=10)
#ldamodel.save(filename+'_title.gensim')
topics = ldamodel_title.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.026*"opioid" + 0.021*"analgesic" + 0.013*"patient" + 0.010*"morphine" + 0.010*"postoperative"')
(1, '0.052*"sickle" + 0.040*"disease" + 0.038*"opioid" + 0.030*"analgesic" + 0.030*"patient"')
(2, '0.064*"opioid" + 0.040*"sickle" + 0.037*"analgesic" + 0.027*"acute" + 0.021*"disease"')
(3, '0.069*"opioid" + 0.054*"analgesic" + 0.021*"sickle" + 0.020*"prescription" + 0.017*"disease"')
(4, '0.073*"sickle" + 0.062*"disease" + 0.044*"opioid" + 0.038*"management" + 0.025*"analgesic"')


In [21]:
from gensim.models import CoherenceModel# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel_title, texts=title_text_data, dictionary=dictionary_title, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.35098178200889363


In [17]:
best_ldamodel_title = gensim.models.LdaMulticore(corpus_title,num_topics=7, 
                                           random_state=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.90,id2word=dictionary_title)
#ldamodel.save(filepath+filename+'_abstract.gensim')
best_topics = best_ldamodel_title.print_topics(num_words=5)
for topic in best_topics:
    print(topic)

(0, '0.022*"sickle" + 0.019*"disease" + 0.007*"report" + 0.006*"management" + 0.006*"clinical"')
(1, '0.063*"sickle" + 0.050*"disease" + 0.020*"management" + 0.019*"acute" + 0.019*"opioid"')
(2, '0.007*"sickle" + 0.006*"opioid" + 0.006*"receptor" + 0.005*"polymorphism" + 0.004*"cyp2d6"')
(3, '0.010*"opioid" + 0.009*"analgesic" + 0.008*"patient" + 0.005*"clinical" + 0.004*"cancer"')
(4, '0.013*"opioid" + 0.012*"analgesic" + 0.005*"prescription" + 0.005*"effect" + 0.004*"study"')
(5, '0.052*"opioid" + 0.043*"analgesic" + 0.015*"patient" + 0.008*"cancer" + 0.007*"review"')
(6, '0.040*"opioid" + 0.022*"analgesic" + 0.012*"sickle" + 0.011*"patient" + 0.011*"management"')


In [18]:
from gensim.models import CoherenceModel# Compute Coherence Score
best_coherence_model_lda_title = CoherenceModel(model=best_ldamodel_title, texts=title_text_data, dictionary=dictionary_title, coherence='c_v')
best_coherence_lda_title = best_coherence_model_lda_title.get_coherence()
print('\nCoherence Score: ', best_coherence_lda_title)


Coherence Score:  0.4059407043041047


In [22]:
import pyLDAvis.gensim
lda_display_title = pyLDAvis.gensim.prepare(best_ldamodel_title, corpus_title, dictionary_title, sort_topics=False)
pyLDAvis.save_html(lda_display_title, filepath+'title_nicotine_pain_opioid.html')
pyLDAvis.display(lda_display_title)


In [ ]:

model_results_titles = {#'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
}# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=120)
    
    # iterate through validation corpuses
    #for i in range(len(corpus_sets)):
        # iterate through number of topics
    for k in topics_range:
            # iterate through alpha values
        for a in alpha:
                # iterare through beta values
            for b in beta:
                    # get the coherence score for the given parameters
                cv = compute_coherence_values(corpus=corpus_title,data=title_text_data, dictionary=dictionary_title,k=k, a=a, b=b)
                    # Save the model results
                   # model_results['Validation_Set'].append(corpus_title[i])
                model_results_titles['Topics'].append(k)
                model_results_titles['Alpha'].append(a)
                model_results_titles['Beta'].append(b)
                model_results_titles['Coherence'].append(cv)
                    
                pbar.update(1)
    #pd.DataFrame(model_results).to_csv('lda_tuning_query_and_or_opioid.csv', index=False)
    pbar.close()

In [ ]:
pd.DataFrame(model_results_titles).to_csv(filepath+'title_query4_and_or_opioid_0.3.csv', index=False)